### Convert RealSense .bag → MP4

In [ ]:
import numpy as np
import pyrealsense2 as rs
import cv2

def bag_to_mp4(bag_path: str, out_path: str, fps: float = 30.0):
    """
    Read a RealSense .bag file and write out a color-only MP4.
    """
    pipeline = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(bag_path, repeat_playback=False)
    profile = pipeline.start(cfg)

    # get color stream resolution
    color_stream = profile.get_stream(rs.stream.color).as_video_stream_profile()
    w, h = color_stream.width(), color_stream.height()

    # set up VideoWriter (mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    try:
        while True:
            frames = pipeline.wait_for_frames()
            color = frames.get_color_frame()
            if not color:
                continue
            # convert to numpy array (RGB)
            img_rgb = np.asanyarray(color.get_data())
            # convert RGB→BGR for correct colors in OpenCV
            img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
            writer.write(img_bgr)
    except RuntimeError:
        # bag playback ended
        pass
    finally:
        writer.release()
        pipeline.stop()
        print(f"✅ Wrote {out_path}")



### Run conversion

In [2]:
bag_to_mp4(bag_file, output_mp4, fps)

✅ Wrote /home/yogee/Desktop/human_detector_ws/training_data/raw_data/video1.mp4


### Split dataset

In [2]:
# %% [code]
# 1) (Optional) install PyYAML if you haven't already
# !pip install pyyaml --quiet

import shutil
import random
from pathlib import Path
import yaml

# 2) Define your paths (all relative to your notebook cwd == HUMAN_DETECTOR_WS)
repo_root  = Path().resolve()
export_dir = repo_root / "labelled_data" / "Video_1" / "obj_train_data"
names_file = repo_root / "labelled_data" / "Video_1" / "obj.names"
dest_dir   = repo_root / "../training_data"

assert export_dir.exists(), f"Export dir not found: {export_dir}"
assert names_file.exists(), f"Names file not found: {names_file}"

# 3) Create train/val subfolders under training_data/images and training_data/labels
for split in ("train","val"):
    (dest_dir/"images"/split).mkdir(parents=True, exist_ok=True)
    (dest_dir/"labels"/split).mkdir(parents=True, exist_ok=True)

# 4) Gather all image files from the CVAT export
img_exts = {".png", ".jpg", ".jpeg"}
all_imgs = sorted(p for p in export_dir.iterdir() if p.suffix.lower() in img_exts)

# 5) Shuffle & split 80/20
random.seed(42)
random.shuffle(all_imgs)
n_train   = int(0.8 * len(all_imgs))
train_imgs, val_imgs = all_imgs[:n_train], all_imgs[n_train:]

# 6) Copy images and their .txt labels into the training_data folder structure
for split, imgs in [("train", train_imgs), ("val", val_imgs)]:
    for img_path in imgs:
        lbl_path = export_dir / f"{img_path.stem}.txt"
        if not lbl_path.exists():
            continue
        shutil.copy2(img_path,   dest_dir/"images"/split/img_path.name)
        shutil.copy2(lbl_path,   dest_dir/"labels"/split/f"{img_path.stem}.txt")

# 7) Read your class names
with open(names_file) as f:
    names = [line.strip() for line in f if line.strip()]

# 8) Write a data.yaml for YOLO training
cfg = {
    "path": str(dest_dir),     # base dir for train/val
    "train": "images/train",
    "val":   "images/val",
    "nc":    len(names),
    "names": names
}
with open(dest_dir/"data.yaml", "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print(f"✅ Copied {len(train_imgs)} train / {len(val_imgs)} val images")
print(f"✅ data.yaml written to {dest_dir/'data.yaml'}")


✅ Copied 1324 train / 331 val images
✅ data.yaml written to /home/yogee/Desktop/human_detector_ws/raw_data/../training_data/data.yaml
